# Quiz

In [13]:
import pandas as pd
import numpy as np
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
#Loading in dataset
df = pd.read_csv('data/Portfolios_Formed_on_ME_monthly_EW.csv',
                header = 0,
                index_col = 0,
                parse_dates = True)

In [15]:
df = df[['Lo 20', 'Hi 20']]
df = df / 100
df.index = pd.to_datetime(df.index, format = "%Y%m").to_period('M')

In [16]:
df.head()

,Lo 20,Hi 20
1926-07,-0.0057,0.0333
1926-08,0.0384,0.0233
1926-09,-0.0048,-0.0009
1926-10,-0.0329,-0.0295
1926-11,-0.0055,0.0316


## Question 1, 3
What was the Annualized Return of the portfolio over the entire period?

In [17]:
#calculating volatility and anualized volatilty
vol = df.std()
annualized_vol = vol * np.sqrt(12)

#Anualizing Retruns
no_of_months = df.shape[0]
total_returns = (df+1).prod() -1
returns_per_month = (df+1).prod() ** (1/no_of_months) -1

#annualized return = (returns +1).prod() ** (no_of_periods/ no_of_months) -1
annualized_return = (df + 1).prod() ** (12/no_of_months) -1
annualized_return

Lo 20    0.151977
Hi 20    0.098490
dtype: float64

## Question 2, 4
What was the Annualized Volatility of the portfolio over the entire period?

In [18]:
vol = df.std()
annualized_vol = vol * np.sqrt(12)
annualized_vol

Lo 20    0.336701
Hi 20    0.195116
dtype: float64

## Question 5, 7
What was the Annualized Return of the Lo 20 and Hi 20 portfolio over the period 1999 - 2015 (both inclusive)? 

In [26]:
def get_ffme_returns():
    """
    Load the Fama-French Dataset for the returns of the Top and Bottom Deciles by MarketCap
    """
    me_m = pd.read_csv("data/Portfolios_Formed_on_ME_monthly_EW.csv",
                       header=0, index_col=0, na_values=-99.99)
    rets = me_m[['Lo 20', 'Hi 20']]
    rets.columns = ['SmallCap', 'LargeCap']
    rets = rets/100
    rets.index = pd.to_datetime(rets.index, format="%Y%m").to_period('M')
    return rets

def annualize_returns(r, periods_per_year):
    """
    Annualizes a set of returns
    based on periods per year
    """
    compounded_growth = (1+r).prod()
    n_periods = r.shape[0]
    return compounded_growth**(periods_per_year/n_periods)-1

In [27]:
returns = get_ffme_returns()
print(annualize_returns(returns.loc['1999-01':'2015-12'], 12))

SmallCap    0.114426
LargeCap    0.062865
dtype: float64


## Question 6, 8
What was the Annualized Volatilaty of the Lo 20 and Hi 20 portfolio over the period 1999 - 2015 (both inclusive)? 

In [28]:
def annualized_volatility(r, periods_per_year):
    """
    Annualizes the vol of a set of returns
    based onthe periods per year
    """
    return r.std()*(periods_per_year**0.5)

In [29]:
annualVol = annualize_volatility(returns.loc['1999-01':'2015-12'], 12)
print(annualized_vol)

Lo 20    0.336701
Hi 20    0.195116
dtype: float64


# Question 9, 10
What was the Max Drawdown (expressed as a positive number) experienced over the 1999-2015 period in the SmallCap (Lo 20) portfolio?

at the end of which month over the period 1999-2015 did that maximum drawdown on the SmallCap (Lo 20) portfolio occur?

In [30]:
def drawdown(return_series: pd.Series):
    """Takes a time series of asset returns.
       returns a DataFrame with columns for
       the wealth index, 
       the previous peaks, and 
       the percentage drawdown
    """
    wealth_index = 1000*(1+return_series).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks)/previous_peaks
    return pd.DataFrame({"Wealth": wealth_index, 
                         "Previous Peak": previous_peaks, 
                         "Drawdown": drawdowns})

In [36]:
maxDrawdown = drawdown(returns.loc['1999-01':'2015-12', 'SmallCap'])
maxDrawdown[maxDrawdown.Drawdown == min(maxDrawdown.Drawdown)]

,Wealth,Previous Peak,Drawdown
2009-02,1804.729077,4810.164013,-0.624809


# Question 11, 12
What was the Max Drawdown (expressed as a positive number) experienced over the 1999-2015 period in the LargeCap (Hi 20) portfolio?

Over the period 1999-2015, at the end of which month did that maximum drawdown of the LargeCap (Hi 20) portfolio occur? 

In [37]:
maxDrawdownHi = drawdown(returns.loc['1999-01':'2015-12', 'LargeCap'])
maxDrawdownHi[maxDrawdownHi.Drawdown == min(maxDrawdownHi.Drawdown)]

,Wealth,Previous Peak,Drawdown
2009-02,779.227076,1742.204449,-0.552735


# Question 13, 14
For the remaining questions, use the EDHEC Hedge Fund Indices data set. 

Looking at the data since 2009 (including all of 2009) through 2018 which Hedge Fund Index has exhibited the highest and lowest semideviation?    

In [38]:
def get_hfi_returns():
    """
    Load and format the EDHEC Hedge Fund Index Returns
    """
    hfi = pd.read_csv("data/edhec-hedgefundindices.csv",
                      header=0, index_col=0, parse_dates=True)
    hfi = hfi/100
    hfi.index = hfi.index.to_period('M')
    return hfi

In [39]:
def semideviation(r):
    """
    Returns the semideviation aka negative semideviation of r
    r must be a Series or a DataFrame, else raises a TypeError
    """
    if isinstance(r, pd.Series):
        is_negative = r < 0
        return r[is_negative].std(ddof=0)
    elif isinstance(r, pd.DataFrame):
        return r.aggregate(semideviation)
    else:
        raise TypeError("Expected r to be a Series or DataFrame")

In [41]:
hfi = get_hfi_returns()
semiDeviation = semideviation(hfi.loc['2009-01':, :])
semiDeviation.sort_values(ascending=False)

/Users/rishik/opt/anaconda3/lib/python3.9/site-packages/pandas/io/parsers/base_parser.py:1070: UserWarning: Parsing '31/01/1997' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return tools.to_datetime(
/Users/rishik/opt/anaconda3/lib/python3.9/site-packages/pandas/io/parsers/base_parser.py:1070: UserWarning: Parsing '28/02/1997' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return tools.to_datetime(
/Users/rishik/opt/anaconda3/lib/python3.9/site-packages/pandas/io/parsers/base_parser.py:1070: UserWarning: Parsing '31/03/1997' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return tools.to_datetime(
/Users/rishik/opt/anaconda3/lib/python3.9/site-packages/pandas/io/parsers/base_parser.py:1070: UserWarning: Parsing '30/04/1997' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  re

Short Selling             0.021240
Emerging Markets          0.015276
Long/Short Equity         0.012139
CTA Global                0.010996
Event Driven              0.010135
Distressed Securities     0.009498
Funds Of Funds            0.007948
Convertible Arbitrage     0.006681
Relative Value            0.005711
Equity Market Neutral     0.005186
Global Macro              0.005075
Merger Arbitrage          0.003611
Fixed Income Arbitrage    0.003321
dtype: float64

# Question 15
Looking at the data since 2009 (including all of 2009) which Hedge Fund Index has been most negatively skewed? 

In [42]:
def skewness(r):
    """
    Alternative to scipy.stats.skew()
    Computes the skewness of the supplied Series or DataFrame
    Returns a float or a Series
    """
    demeaned_r = r - r.mean()
    # use the population standard deviation, so set dof=0
    sigma_r = r.std(ddof=0)
    exp = (demeaned_r**3).mean()
    return exp/sigma_r**3

In [43]:
skewness(hfi.loc['2009-01':,:]).sort_values()

Equity Market Neutral    -0.896327
Funds Of Funds           -0.646908
Merger Arbitrage         -0.551065
Event Driven             -0.488821
Long/Short Equity        -0.463703
Distressed Securities    -0.254944
Emerging Markets          0.033123
CTA Global                0.052062
Relative Value            0.159953
Global Macro              0.348184
Short Selling             0.456518
Fixed Income Arbitrage    1.121453
Convertible Arbitrage     1.305911
dtype: float64

# Question 16
Looking at the data since 2000 (including all of 2000) through 2018 which Hedge Fund Index has exhibited the highest kurtosis?

In [44]:
def kurtosis(r):
    """
    Alternative to scipy.stats.kurtosis()
    Computes the kurtosis of the supplied Series or DataFrame
    Returns a float or a Series
    """
    demeaned_r = r - r.mean()
    # use the population standard deviation, so set dof=0
    sigma_r = r.std(ddof=0)
    exp = (demeaned_r**4).mean()
    return exp/sigma_r**4

In [45]:
kurtosis(hfi.loc['2000-01':,:]).sort_values()

CTA Global                 2.845022
Global Macro               3.653278
Short Selling              4.352758
Long/Short Equity          4.472589
Merger Arbitrage           4.520785
Event Driven               5.543444
Emerging Markets           5.941289
Distressed Securities      6.231720
Funds Of Funds             7.584725
Relative Value            12.731096
Equity Market Neutral     20.329621
Convertible Arbitrage     23.001449
Fixed Income Arbitrage    33.378729
dtype: float64